In [49]:
import numpy as np 
import random 
from network import Network
from sklearn.utils.extmath import softmax

def _create_population(size=100):
    """
    size: population size, number of chromesomes 

    return population, which is a list of random created network graphs
    """
    population = []
    for _ in range(size):
        nn = Network()
        nn.random_initialize() #  shape 
        population.append(nn)
    return population

def encode(weights, bias):
    """
    encode genes to a chromosome
    """
    gene=[]
    assert len(weights)==len(bias), "length of weiths not equal to length of bias"
    n_layers = len(bias)
    for i in range(n_layers):
        gene.extend(weights[k].flatten())
        gene.append(bias[i])
    return gene 
         

    
def decode(gene,structure):
    """
    decoed genes of a chromosome
    structure is the network structure, e.g.[num_inputs, 10,20,30]
    """
    Weights ={}
    Bias={}
    
    gene_index = 0
    for index in range(len(structure)):
        weight=np.zeros((structure[index],structure[index+1]))
        bias =[]
        for i in range(structure[index]):
            for j in range(structure[index+1]):
                weight[i][j] = gene[gene_index]
                gene_index+=1
        bias.append(gene[gene_index])
        gene_index+=1
        Weights[index] = weight
        Bias[index] = bias
        
    return Weights, Bias
           
    
def mutate(chro, mutation_rate):
    #for each gene, it can decide to change itself to the mean of its nearby genes or not
    new_chro = []
        
    for i in range(len(chro)-1):
        xx = (chro[i]+chro[i-1]+chro[i+1] )/3
        new_chro.append(xx)
    return new_chro
    

def crossover(chro1, chro2):
    # create a random crossover point
    assert len(chro1) ==len(chro2), "chromosome not the same length"
    index = np.random.randint(low=1, high=len(chro1))
    temp1 = chro1[index:]
    temp3 = chro1[:index]
    temp2 = chro2[:index]
    temp4 = chro2[index:]
    #random combine half of chro1 genes and half of chro2 genes below
    rd = np.random.choice([1,2,3,4])
    if rd ==1:
        new_chro = temp1.extend(temp2)
    if rd ==2:
        new_chro = temp1.extend(temp4)
    if rd ==3:
        new_chro = temp3.extend(temp2)
    if rd==4:
        new_chro = temp3.extend(temp4)
    return new_chro
    

def selection(parents,pop):
    chirldren=[] #get two chirldren
    chrome={}
    for i in range(len(parents)):
        parent_id = parents[i][0]
        nn=pop[parent_id]
        weight = nn.getWeights()
        bias = nn.getBias()
        gene = encode(weights, bias)
        chrome[i] = gene
    
    chirldren[0] = crossover(chrome[0], chrome[1])
    chirldren[1] = crossover(chrome[2],chrome[3])
    
    return chirldren
           
         
def GeneticAlgorithmTrainner(X,y, nn_structure, episodes = 10000, mutation_rate = 0.1):
    # first generation 
    pop = _create_population()
    fit_score= []
    
    avg_fit_scores=[]
    for i in range(len(pop)):
        nn = pop[i]
        score = fitness(nn,X,y)
        fit_score.append([i,score])
    
    fit_score=np.array(fit_score)
    fit_score[np.argsort(fit_score[:, 1])]
    fit_score=np.flip(fit_score,0)
    
    len_pop=[]
    
    # select the nn with hightest 4 fitness scores
    for epi in range(episodes):
        len_pop.append(len(pop))
        if len_pop[epi] ==len_pop[epi-1]:
            #converged 
            print("converged")
            break 
        
        avg_fit_scores.append(mean(fit_score[:,1]))
        
        luckyest = fit_score[:4]
        
        #remove two worst case
#         worst2= fit_score[-2:]
#         for s in worst2:
#             del pop[s[0]]
        
        offspring = selection(luckyest, pop)
        
        for kid in offspring:
            #mutation
            p = np.random.random_sample()
            if p <mutation_rate:
                kid = mutate(kid, mutation_rate )
            Weights, Bias = decode(kid)
            nn = Network()
            nn.setWeights(Weights)
            nn.setBias(Bias)
            pop.append(nn)
        fit_score= []
        #calculate new fitness score 
        for i in range(len(pop)):
            nn = pop[i]
            score = fitness(nn,X,y)
            fit_score.append([i,score])
        fit_score=np.array(fit_score)
        fit_score[np.argsort(fit_score[:, 1])]
        fit_score=np.flip(fit_score,0)
        
        len_pop = len(population)
        if len_pop
    return best_fit_scores
        
def fitness(nn, X, y):
    ## 1/ cross_entropy 
    W = nn.getWeights()  #W is a dict, W[i] = [[n_layer_nodes, n-1_layer_nodes]]
    b = nn.getBias() # b is a dict, b[i] = [bias of layer n]
    z=[]
    a = []
    probs = []
    n_layer = len(W)
    
    for i in range(n_layer):
        if i ==0: 
            z = X.dot(W[i])+b[i]
            a = np.tanh(z)
        elif i< n_layer-1:
            z = a.dot(W[i])+b[i]
            a = np.tanh(z)
        else: # softmax
            z = a.dot(W[i])+b[i]
            probs = softmax(z)
    # use softmax to calculate cross entropy below:
    
    m = y.shape[0]
    log_likelihood = -np.log(probs[range(m),y])
    loss = np.sum(log_likelihood) / m
    fit = 1/loss
    return fit
    
                

In [32]:
#load data below

with open('seeds_dataset.txt', 'r') as f:
    x = f.readlines()
data=[]

for line in x:
    temp=[]
    temp.append(line.strip().split())
    l = []
    
    for ele in temp[0]:
        l.append(float(ele))
    data.append(l)
#     print(l)
data = np.array(data)
np.random.shuffle(data)
print(data)

[[14.46   14.35    0.8818 ...  2.802   5.044   1.    ]
 [11.23   12.88    0.8511 ...  4.325   5.003   3.    ]
 [19.51   16.71    0.878  ...  2.962   6.185   2.    ]
 ...
 [12.19   13.2     0.8783 ...  3.631   4.87    3.    ]
 [14.79   14.52    0.8819 ...  2.704   5.111   1.    ]
 [14.43   14.4     0.8751 ...  3.975   5.144   1.    ]]


In [40]:
train_X = []
train_y = []
test_X=[]
test_y=[]
len_data = len(data)
train_data = data[:int(len_data*0.8)]
test_data = data[int(len_data*0.8):]
train_X = np.array(train_data[:, [column for column in range(len(train_data[0])-1)]])
test_X  = np.array(test_data[:, [column for column in range(len(train_data[0])-1)]])
train_y = np.array([train_data[:,len(train_data[0])-1 ]])
test_y = np.array([test_data[:,len(train_data[0])-1 ]])

print(train_X.shape)
print(test_y.shape)

(168, 7)
(1, 42)


In [50]:
fit_data = GeneticAlgorithmTrainner(train_X,train_y, )

1
